use newpycaret

In [1]:
# packages
import snowflake.connector
import pandas as pd
import os
import numpy as np
from autots import AutoTS
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime, timedelta
from dateutil import relativedelta

2. Load Data
---
### To Do: copy and paste in to a new chunk, enter credentials and run to save in environment. Then delete chunk

%env snowflakeuser=<your_snowflake_username> <br>
%env snowflakepass=<your_snowflake_password>

In [3]:
# Query Snowflake

# Snowflake connection parameters
connection_params = {
    "user": os.environ['snowflakeuser'],
    "password": os.environ['snowflakepass'],
    "account": "zib52348.us-east-1",
    "role": "ACCOUNTADMIN",
    "warehouse": "REPORTING",
    "database": "ANALYTICS",
    "schema": "FORECASTING",
}

# Establish a connection to Snowflake
conn = snowflake.connector.connect(**connection_params)

with open('net_sales_query.sql', 'r') as query:
    # connection == the connection to your database, in your case prob_db
    df_initial = pd.read_sql_query(query.read(), conn)

# Close the connection
conn.close()

C:\Users\crudek\AppData\Local\Temp\ipykernel_3952\2623517584.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_initial = pd.read_sql_query(query.read(), conn)


In [16]:
##### INITIAL VARIABLE DECLARATION #####

# Get the current date
current_date = datetime.today()

# Calculate the first day of the current month
first_day_of_current_month = pd.to_datetime(current_date.replace(day=1))

# Calculate the first day of the next month
first_day_of_next_month = datetime(current_date.year, current_date.month + 1, 1)

# Calculate the first day of the last month
last_day_of_last_month = first_day_of_current_month - timedelta(days=1)
first_day_of_last_month = last_day_of_last_month.replace(day=1)




### MANUAL INPUTS ###

# how much data do you want to keep to train models. 
#end_of_data = first_day_of_last_month # the default which is the max date this should ever be because it would include all months that have full data
end_of_data = pd.to_datetime('2022-12-01')

# forecast horizon = how many months in to the future you want to forecast. So, we will forecast this many months past the above end_of_data
fh = 12




### FORECAST HORIZON (FH) DATAFRAME ###

end_of_data_next_month = end_of_data + relativedelta.relativedelta(months=1, day=1)
end_of_data_df = pd.DataFrame({'end_of_data': [end_of_data_next_month]})
end_of_data_df['end_of_data'] = pd.to_datetime(end_of_data_df['end_of_data'])

# Create a date range for the next 12 months
next_12_months = pd.date_range(start=end_of_data_df['end_of_data'].iloc[0], periods=fh, freq='MS')

fh_dates_df = pd.DataFrame({'MONTH': next_12_months})




### INITIAL DATA PREPARATION ###

# create copy of df_d
df = df_initial.copy(deep=True)

# convert month field to date
df["MONTH"] = pd.to_datetime(df["MONTH"])

# Some random months will have data that we want to remove (* Want to test without July though)
df = df[["DEP_ENT", "MONTH", "NET_SALES"]]  # select fields of interest
df = df.sort_values(['DEP_ENT', 'MONTH'])  # reorder dataframe

# remove data after the 'end of data' setting above
df_sub = df[df['MONTH'] <= end_of_data]

# create series
df_s = df_sub.set_index(['DEP_ENT','MONTH'])['NET_SALES']
# convert back to dataframe
df_d = df_s.to_frame()
#reset index
df_d.reset_index(inplace=True)




### IMPUTE MISSIG VALUES WITH MONTH AND MEAN ###

# list of each dep-ent
all_dep_ent = df_d['DEP_ENT'].unique()

# Create empty dataframe
df_subset_all = pd.DataFrame()

for i in tqdm(all_dep_ent):
    
    temp_subset = df_d[df_d['DEP_ENT'] == i]
    
    # Define the minimum and maximum dates
    min_date = min(temp_subset['MONTH'])
    max_date = end_of_data # max(temp_subset['MONTH'])

    # Generate a list of dates for each month in between
    date_range = []
    current_month = min_date.replace(day=1)
    while current_month <= max_date:
        date_range.append(current_month)
        current_month = current_month + timedelta(days=32)
        current_month = current_month.replace(day=1)

    # Create a DataFrame from the list of dates
    date_range_df = pd.DataFrame({'MONTH': date_range})

    df_merged = pd.merge(date_range_df, temp_subset, on='MONTH', how='left')

    #Finding the mean of the column having NaN
    mean_value = df_merged['NET_SALES'].mean()
    
    # Replace NaNs in column S2 with the
    # mean of values in the same column
    df_merged['NET_SALES'].fillna(value=mean_value, inplace=True)
    
    df_merged['DEP_ENT'] = i

    df_subset_all = pd.concat([df_subset_all,df_merged])

    


### FILL DATES THROUGH END OF FORECAST HORIZON ###

# Create empty dataframe
df_subset_all2 = pd.DataFrame()

for i in tqdm(all_dep_ent):
    # filter to one dep_ent
    temp_subset2 = df_subset_all[df_subset_all['DEP_ENT'] == i]
    # temp result
    temp_result2 = pd.merge(temp_subset2, fh_dates_df, on='MONTH', how='outer')
    temp_result2['DEP_ENT'] = i
    
    # combine for all dep_ents
    df_subset_all2 = pd.concat([df_subset_all2,temp_result2])

100%|██████████| 14/14 [00:00<00:00, 624.38it/s]


In [17]:
#from autots.models.model_list import model_lists
#print(model_lists['univariate'])

['NeuralProphet', 'ETS', 'GLS', 'SeasonalNaive', 'UnivariateMotif', 'UnivariateRegression', 'ARCH', 'LastValueNaive', 'MetricMotif', 'MLEnsemble', 'Theta', 'DatepartRegression', 'ARDL', 'SeasonalityMotif', 'GLM', 'AverageValueNaive', 'ConstantNaive', 'UnobservedComponents', 'ARIMA', 'FBProphet', 'KalmanStateSpace']


In [18]:
### Time Series Loop ###

# Create empty dataframes
all_predictions = pd.DataFrame()

# list of each dep-ent
de_list = df_subset_all2['DEP_ENT'].unique()

for i in tqdm(de_list):

    df_subset = df_subset_all2[df_subset_all2['DEP_ENT'] == i]
    df_subset_trim = df_subset[df_subset['MONTH'] <= end_of_data]
    df_subset_trim = df_subset_trim[['MONTH', 'NET_SALES']]
    df_subset_trim.reset_index()

    # model
    model = AutoTS(
        forecast_length = fh,
        frequency = 'MS',
        ensemble = 'simple', 
        models_mode = 'deep',
        model_list = 'univariate',
        max_generations = 3,
        num_validations = 3,
        no_negatives = True,
        n_jobs = 'auto'
    )

    model = model.fit(
        df_subset_trim,
        date_col = 'MONTH',
        value_col = 'NET_SALES',
    )

    # print(model)

    # create prediction
    prediction = model.predict(forecast_length=fh)

    # temp fcast dataframe
    temp_fcasts = prediction.forecast

    # make index, date, a column
    temp_fcasts_ni = temp_fcasts.reset_index()

    # rename
    temp_fcasts_ni.rename(columns={'index': 'MONTH'}, inplace=True)

    temp_fcasts_ni['DEP_ENT'] = i  # add dep

    # append to master dataframe
    all_predictions = all_predictions.append(temp_fcasts_ni)

  0%|          | 0/14 [00:00<?, ?it/s]

Using 6 cpus for n_jobs.
Data frequency is: MS, used frequency is: MS
Model Number: 1 with model ARIMA in generation 0 of 3
Model Number: 2 with model AverageValueNaive in generation 0 of 3
Model Number: 3 with model AverageValueNaive in generation 0 of 3
Model Number: 4 with model AverageValueNaive in generation 0 of 3
Model Number: 5 with model DatepartRegression in generation 0 of 3
Model Number: 6 with model DatepartRegression in generation 0 of 3
Model Number: 7 with model DatepartRegression in generation 0 of 3


c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Model Number: 8 with model DatepartRegression in generation 0 of 3
Template Eval Error: ImportError('Tensorflow not available, install with pip install tensorflow.') in model 8 in generation 0: DatepartRegression
Model Number: 9 with model ETS in generation 0 of 3
Model Number: 10 with model ETS in generation 0 of 3
Model Number: 11 with model GLM in generation 0 of 3
Model Number: 12 with model GLM in generation 0 of 3


c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


...

WARNING - (py.warnings._showwarnmsg) - c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\autots\tools\probabilistic.py:67: RuntimeWarning: invalid value encountered in divide
  (prior_mu / prior_sigma**2) + ((n * data_mu) / prior_sigma**2)

WARNING - (py.warnings._showwarnmsg) - c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\autots\tools\probabilistic.py:68: RuntimeWarning: divide by zero encountered in divide
  ) / ((1 / prior_sigma**2) + (n / prior_sigma**2))

WARNING - (py.warnings._showwarnmsg) - c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:307: DomainWarning: The InversePower link function does not respect the domain of the Gamma family.
  warnings.warn((f"The {type(family.link).__name__} link function "

WARNING - (py.warnings._showwarnmsg) - c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\statsmodels\genmod\families\family.py:18

53 - DatepartRegression with avg smape 56.19: 
Model Number: 54 of 71 with model SeasonalityMotif for Validation 2
Template Eval Error: Exception('Transformer Detrend failed on fit') in model 54 in generation 0: SeasonalityMotif
Model Number: 55 of 71 with model LastValueNaive for Validation 2
Template Eval Error: Exception('Transformer AlignLastValue failed on fit') in model 55 in generation 0: LastValueNaive
Model Number: 56 of 71 with model GLM for Validation 2
Template Eval Error: ValueError('The first guess on the deviance function returned a nan.  This could be a boundary  problem and should be reported.') in model 56 in generation 0: GLM
Model Number: 57 of 71 with model DatepartRegression for Validation 2
57 - DatepartRegression with avg smape 49.6: 
Model Number: 58 of 71 with model UnivariateMotif for Validation 2
Template Eval Error: ValueError('window shape cannot be larger than input array shape') in model 58 in generation 0: UnivariateMotif
Model Number: 59 of 71 with mod

WARNING - (py.warnings._showwarnmsg) - c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\autots\tools\probabilistic.py:67: RuntimeWarning: invalid value encountered in divide
  (prior_mu / prior_sigma**2) + ((n * data_mu) / prior_sigma**2)

WARNING - (py.warnings._showwarnmsg) - c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\autots\tools\probabilistic.py:68: RuntimeWarning: divide by zero encountered in divide
  ) / ((1 / prior_sigma**2) + (n / prior_sigma**2))



6 - Ensemble with avg smape 50.76: 
Model Number: 7 of 8 with model Ensemble for Validation 2
7 - Ensemble with avg smape 49.5: 
Model Number: 8 of 8 with model Ensemble for Validation 2
8 - Ensemble with avg smape 49.17: 


WARNING - (py.warnings._showwarnmsg) - C:\Users\crudek\AppData\Local\Temp\ipykernel_3952\1526884490.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_predictions = all_predictions.append(temp_fcasts_ni)

100%|██████████| 14/14 [1:01:39<00:00, 264.23s/it]


In [19]:
### AFTER THE LOOP ###

# rename index col
all_predictions.rename(columns={'index': 'MONTH', 'NET_SALES': 'PRED'}, inplace=True)

# initial sales pull
sales = df_initial
# select cols
sales = sales[["DEP_ENT", "MONTH", "NET_SALES"]]
sales["MONTH"] = pd.to_datetime(sales["MONTH"])
# only include sales data for the full months we have
sales = sales[sales['MONTH'] < pd.to_datetime(first_day_of_current_month)]

# combine prediction data and original sales data
merged = pd.merge(all_predictions, sales, how='left', on=['DEP_ENT', 'MONTH'])

WARNING - (py.warnings._showwarnmsg) - C:\Users\crudek\AppData\Local\Temp\ipykernel_3952\250935752.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales["MONTH"] = pd.to_datetime(sales["MONTH"])



In [20]:
### BUDGET ###

# Establish a connection to Snowflake
conn = snowflake.connector.connect(**connection_params)

with open('budget_query.sql', 'r') as query:
    # connection == the connection to your database, in your case prob_db
    budg = pd.read_sql_query(query.read(), conn)

# Close the connection
conn.close()

# convert month to datetime
budg["MONTH"] = pd.to_datetime(budg["MONTH"])
# select cols
budg = budg[["MONTH", "BUDGET_AMOUNT", "DEP_ENT"]]

WARNING - (py.warnings._showwarnmsg) - C:\Users\crudek\AppData\Local\Temp\ipykernel_3952\690824450.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  budg = pd.read_sql_query(query.read(), conn)



In [21]:
# combine prediction/sales data with budget data
merged2 = pd.merge(merged, budg, how='left', on=['DEP_ENT', 'MONTH'])

# write csv
merged2.to_csv('univariate_pred.csv')